In [544]:
#Imports
import spotipy, json, webbrowser, urllib.request
import pandas as pd
import spotipy.util as util
import random
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate




In [545]:
#User Input Cell 1
kseniia_lucky_number = 5
omotara_lucky_number = 3

#Manual input, comment the below lines out when ready
user_input = [["Dynamic", "Spicy"],  ["Beef"], ["Thai", "Indian", "Korean"], [28, 11, 23]]
date_vibe = user_input[0]
dietary_req = user_input[1]
pref_cuisines = user_input[2]
date_day = user_input[3][0]
date_month = user_input[3][1]
date_year = user_input[3][2]

#Dynamic input, uncomment the below lines later
# true_date_vibe = input("Tell me about your date... \n What vibe do you want? \n Describe it in a list (e.g. ['Dynamic', 'Spicy'])")
# true_dietary_req = input("And what are your dietary requirements? \n Give them in a list (e.g. ['Beef'])\n")
# true_pref_cuisines = input("What are your preferred cuisines? \n Write them as a list (e.g. ['Italian', 'Korean'])\n")
# true_date = input("And finally when is the date? \n Format it like this [dd, mm, yy] (e.g. [28, 11, 23])\n")
# date_day = true_date[0]
# date_month = true_date[1]
# date_year = true_date[2]

In [546]:
#Cocktail Cell 1
letters = []

for word in date_vibe:
    for char in word:
        letters.append(char.lower())

print(letters)

['d', 'y', 'n', 'a', 'm', 'i', 'c', 's', 'p', 'i', 'c', 'y']


In [547]:
#Cocktail Cell 2
cocktail_suggestions = []
for letter in letters:
    number = kseniia_lucky_number
    query_str = f"http://www.thecocktaildb.com/api/json/v1/1/search.php?f={letter}"
    request = urllib.request.Request(query_str)
    response = urllib.request.urlopen(request)
    all_cocktails = json.loads(response.read())
    if len(all_cocktails['drinks']) < (number-1):
        number = len(all_cocktails['drinks'])
    cocktail = all_cocktails['drinks']
    if cocktail[0]['strDrink'] not in cocktail_suggestions:
        cocktail_name = cocktail[0]['strDrink']
        cocktail_instructions = cocktail[0]['strInstructions']
        cocktail_ingred = cocktail[0]['strIngredient1'].lower()
        cocktail_img = cocktail[0]['strDrinkThumb']
        num = 2
        while cocktail[0][f'strIngredient{num}'] != None:
            cocktail_ingred = cocktail_ingred + ", " + cocktail[0][f'strIngredient{num}'].lower()
            num += 1
        cocktail_suggestions.append([cocktail_name, cocktail_instructions, cocktail_ingred, cocktail_img])
print(cocktail_suggestions)

[['Derby', 'Pour all ingredients into a mixing glass with ice. Stir. Strain into a cocktail glass. Garnish with a sprig of fresh mint in the drink.', 'gin, peach bitters, mint', 'https://www.thecocktaildb.com/images/media/drink/52weey1606772672.jpg'], ['Yellow Bird', 'Shake and strain into a chilled cocktail glass', 'white rum, galliano, triple sec, lime juice', 'https://www.thecocktaildb.com/images/media/drink/2t9r6w1504374811.jpg'], ['Negroni', 'Stir into glass over ice, garnish and serve.', 'gin, campari, sweet vermouth', 'https://www.thecocktaildb.com/images/media/drink/qgdu971561574065.jpg'], ['A1', 'Pour all ingredients into a cocktail shaker, mix and serve over ice into a chilled glass.', 'gin, grand marnier, lemon juice, grenadine', 'https://www.thecocktaildb.com/images/media/drink/2x8thr1504816928.jpg'], ['Melya', 'Brew espresso. In a coffee mug, place 1 teaspoon of unsweetened powdered cocoa, then cover a teaspoon with honey and drizzle it into the cup. Stir while the coffee 

In [548]:
#Cocktail Cell 3
number = omotara_lucky_number
random.shuffle(cocktail_suggestions)
if len(cocktail_suggestions) < number:
    number = len(cocktail_suggestions)
your_cocktail = cocktail_suggestions[number][0]
your_recipe = cocktail_suggestions[number][1]
your_ingredients = cocktail_suggestions[number][2]
your_cocktail_img = cocktail_suggestions[number][3]

a_or_an = "a"
vowels = ["a", "e", "i", "o", "u"]
if your_cocktail[0].lower() in vowels:
    a_or_an = "an"
    
print(f"Your cocktail for the evening is {a_or_an}\033[1m {your_cocktail}\n\033[0m Here's the recipe \n Ingredients: {your_ingredients}\n Steps: {your_recipe}")

Your cocktail for the evening is a Casino
 Here's the recipe 
 Ingredients: gin, maraschino liqueur, lemon juice, orange bitters, cherry
 Steps: Pour all ingredients into shaker with ice cubes. Shake well. Strain into chilled cocktail glass. Garnish with a lemon twist and a maraschino cherry. Serve without a straw.


In [580]:
#Meal Cell 1

cuisine_suggestions = []
valid_recipes = []

for user_cuisine in pref_cuisines:
    query_str = f"https://www.themealdb.com/api/json/v1/1/filter.php?a={user_cuisine}"
    request = urllib.request.Request(query_str)
    response = urllib.request.urlopen(request)
    all_responses_for_cuisine = json.loads(response.read())
    all_meals_for_cuisine = all_responses_for_cuisine['meals']
    if all_meals_for_cuisine != None:
        for dish in all_meals_for_cuisine:
            dish_id = dish['idMeal']
            query_str = f"https://www.themealdb.com/api/json/v1/1/lookup.php?i={dish_id}"
            request = urllib.request.Request(query_str)
            response = urllib.request.urlopen(request)
            all_dish_responses = json.loads(response.read())
            dish_info = all_dish_responses['meals']
            
            for req in dietary_req:
                if dish_info[0]['strCategory'] == req:
                    meal = dish_info[0]['strMeal']
                    category = dish_info[0]['strCategory']
                    area = dish_info[0]['strArea']
                    instruct = dish_info[0]['strInstructions']
                    image = dish_info[0]['strMealThumb']
                    meal_ingred = dish_info[0]['strIngredient1'].lower()
                    num = 2
                    while dish_info[0][f'strIngredient{num}'] != None:
                        meal_ingred = meal_ingred + ", " + dish_info[0][f'strIngredient{num}'].lower()
                        num += 1
                    ls = [meal, category, area, instruct, image, meal_ingred]
                    valid_recipes.append(ls)
    else:
        print(f"There were no meals for the {user_cuisine} cuisine")
                    
if len(valid_recipes) == 0:
    print("No recipes found for any of your preferred cuisines :( Please try another!")
else: 
    your_meal = random.choice(valid_recipes)
    print(your_meal)
    print(len(your_meal))

There were no meals for the Korean cuisine
['Massaman Beef curry', 'Beef', 'Thai', 'Heat oven to 200C/180C fan/gas 6, then roast the peanuts on a baking tray for 5 mins until golden brown. When cool enough to handle, roughly chop. Reduce oven to 180C/160C fan/gas 4.\r\nHeat 2 tbsp coconut cream in a large casserole dish with a lid. Add the curry paste and fry for 1 min, then stir in the beef and fry until well coated and sealed. Stir in the rest of the coconut with half a can of water, the potatoes, onion, lime leaves, cinnamon, tamarind, sugar, fish sauce and most of the peanuts. Bring to a simmer, then cover and cook for 2 hrs in the oven until the beef is tender.\r\nSprinkle with sliced chilli and the remaining peanuts, then serve straight from the dish with jasmine rice.', 'https://www.themealdb.com/images/media/meals/tvttqv1504640475.jpg', 'peanuts, coconut cream, massaman curry paste, beef, potatoes, onion, lime, cinnamon stick, tamarind paste, brown sugar, fish sauce, chilli, ja

In [581]:
#Meal Cell 2
your_meal_name = your_meal[0]
your_meal_recipe = your_meal[3]
your_meal_img = your_meal[4]
your_meal_ingredients = your_meal[5]

a_or_an_meal = "a"
vowels = ["a", "e", "i", "o", "u"]
if your_meal[0].lower() in vowels:
    a_or_an_meal = "an"

print(your_meal)

['Massaman Beef curry', 'Beef', 'Thai', 'Heat oven to 200C/180C fan/gas 6, then roast the peanuts on a baking tray for 5 mins until golden brown. When cool enough to handle, roughly chop. Reduce oven to 180C/160C fan/gas 4.\r\nHeat 2 tbsp coconut cream in a large casserole dish with a lid. Add the curry paste and fry for 1 min, then stir in the beef and fry until well coated and sealed. Stir in the rest of the coconut with half a can of water, the potatoes, onion, lime leaves, cinnamon, tamarind, sugar, fish sauce and most of the peanuts. Bring to a simmer, then cover and cook for 2 hrs in the oven until the beef is tender.\r\nSprinkle with sliced chilli and the remaining peanuts, then serve straight from the dish with jasmine rice.', 'https://www.themealdb.com/images/media/meals/tvttqv1504640475.jpg', 'peanuts, coconut cream, massaman curry paste, beef, potatoes, onion, lime, cinnamon stick, tamarind paste, brown sugar, fish sauce, chilli, jasmine rice']


In [550]:
#Playlist

In [596]:
#PDF
top_line_pos = 750
line_width = 20
date_details = "28 Nov"
date_canvas = canvas.Canvas(f"date_results/{date_details.replace(" ", "_")}_date_night.pdf", pagesize=A4)
date_canvas.drawString(100, top_line_pos, "MENU")
test = "aygyuwgefkjhdkjhkskjdewkjhd"

def line_y_position(line_idx):
    return top_line_pos-(line_idx*line_width)

def line_break_str(your_string):
    line = ""
    full_str = []
    line_len = 80
    x = line_len
    if line_len < len(your_string):
        first_space = your_string[(line_len-10):(line_len+10)].index(" ") + line_len - 10
        line = your_string[0:first_space]
        full_str.append(line)
        x = first_space
    else:
        line = your_string

    while x < len(your_string):
        y = x+line_len
        if y < len(your_string):
            index_nearest_space = your_string[(y-10):(y+10)].index(" ") + y - 10
        else:
            index_nearest_space = len(your_string)
        line = your_string[x:index_nearest_space]
        x = index_nearest_space
        full_str.append(line)
    
    return full_str

#Cocktail Section
cocktail_img_item = ImageReader(your_cocktail_img)
position = 1
date_canvas.drawString(100, line_y_position(position), f"Your cocktail for the evening is {a_or_an} {your_cocktail}")
position += 1
date_canvas.drawImage(cocktail_img_item, 100, line_y_position(position + 5), mask='auto', width=100, height=100)
position += 7
date_canvas.drawString(100, line_y_position(position), f"Ingredients: {your_ingredients}")
position += 1
date_canvas.drawString(100, line_y_position(position), f"Steps:")
position += 1

cocktail_recipe_broken =  line_break_str(your_recipe)
for line in cocktail_recipe_broken:
    date_canvas.drawString(100, line_y_position(position), f"{line}")
    position += 1

#Meal Section
meal_img_item = ImageReader(your_meal_img)
position += 2
date_canvas.drawString(100, line_y_position(position), f"Your meal for the evening is {a_or_an_meal} {your_meal_name}")
position += 1
date_canvas.drawImage(meal_img_item, 100, line_y_position(position + 5), mask='auto', width=100, height=100)
position += 7
date_canvas.drawString(100, line_y_position(position), f"Ingredients:")
position += 1

meal_ingredients_broken =  line_break_str(your_meal_ingredients)
for line in meal_ingredients_broken:
    date_canvas.drawString(100, line_y_position(position), f"{line}")
    position += 1

date_canvas.drawString(100, line_y_position(position), f"Steps:")
position += 1

meal_recipe_broken =  line_break_str(your_meal_recipe)
for line in meal_recipe_broken:
    date_canvas.drawString(100, line_y_position(position), f"{line}")
    position += 1

#QR Section

#Run
date_canvas.showPage()

date_canvas.drawString(100, line_y_position(1), f"Ingredients:")
position += 1
date_canvas.showPage()
date_canvas.save()